# Introduction to NLP fundamentals

Common NLP problem is sequence to sequence (seq2seq).

In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Visualizing a text dataset

In [7]:
# We want to predict whether the data is a disaster or not a disaster

In [5]:
import pandas as pd

train_df = pd.read_csv("../Datasets/nlp_getting_started/train.csv")
test_df = pd.read_csv("../Datasets/nlp_getting_started/test.csv")

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
# Shuffle our traininig dataframe

train_df_shuffled = train_df.sample(frac=1, random_state=42) #frac is what percentage do we want to shuffle. 1=100%
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
len(train_df), len(test_df)

(7613, 3263)

In [15]:
#Let's visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row # "_" -> to get rid of index
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n {text} \n")
    print("---\n")

Target: 1 (real disaster)
Text:
 Children in Myanmar face a 'double catastrophe' as floods hit the most ... http://t.co/0jFNvAXFph 

---

Target: 0 (not real disaster)
Text:
 Free Ebay Sniping RT? http://t.co/RqIPGQslT6 Chevrolet : Avalanche Ltz Lifted 4x4 Truck ?Please Favorite &amp; Share 

---

Target: 1 (real disaster)
Text:
 Police: Man killed ex grandmother after son wasn&amp;#8217;t named after him http://t.co/ndCy8Q7R6I OMG!!!!! Absolutely Crazy!!! 

---

Target: 1 (real disaster)
Text:
 May Allah help all those suffering from the #Pakistan floods! You and your families are in our #Dua 

---

Target: 1 (real disaster)
Text:
 11:30BST traffic: A10&gt;Paris A40 Geneva A7 Mons A1 Hamburg A2&gt;Hanover A5 Karlsruhe Gotthard n/b  http://t.co/yoi9tOCxiQ 

---



### Split data into Training and Validation set

In [16]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, #10% for validation set
                                                                            random_state = 42)

In [17]:
# Check lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [21]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting Text into Numbers

### Tokenization
Direct mapping of token (word or character) to a number

In [23]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [32]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Default parameters:

text_vectorizer = TextVectorization(max_tokens= None, # how many words in the vocabulary
                                    standardize= "lower_and_strip_punctuation",
                                    split= "whitespace",
                                    ngrams= None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [31]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [34]:
# Setup text vectorization variables

max_vocab_length = 10000
max_length = 15 # How many words of a tweet does a model see

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)